In [17]:
import pandas as pd
from google.oauth2 import service_account
from google.cloud import bigquery

from pycaret.regression import setup, plot_model, create_model

ModuleNotFoundError: No module named 'pycaret'

In [4]:
credentials = service_account.Credentials.from_service_account_file(
    "brasil-aberto-443222-aa742cb8970f.json"
)

client = bigquery.Client("brasil-aberto-443222", credentials=credentials)

In [ ]:
query = """
with enem as (
select
    enem.`CO_MUNICIPIO_ESC`, enem.`Q006`,
    avg(`NU_NOTA_CH`
    + `NU_NOTA_CN`
    + `NU_NOTA_LC`
    + `NU_NOTA_MT`
    + `NU_NOTA_REDACAO`) as nota_media,
from `brasil-aberto-443222.microdados_abertos.enem` as enem
where
    `NU_NOTA_CH`
    + `NU_NOTA_CN`
    + `NU_NOTA_LC`
    + `NU_NOTA_MT`
    + `NU_NOTA_REDACAO`
    > 0
    and enem.`IN_TREINEIRO` != 1
    and enem.`CO_MUNICIPIO_ESC` is not null
group by enem.`CO_MUNICIPIO_ESC`, enem.`Q006`
)

select
  enem.`Q006`,
  enem.nota_media,
  ips.pop,
  ips.pib_pc,
  -- Nutrição e Cuidados Médicos Básicos
  ips.cv_polio,
  ips.hcsap,
  ips.macsap,
  ips.mi5,
  ips.subnutricao,
  -- Abastecimento e Esgotamento
  ips.aavrd,
  ips.esa,
  ips.iaa,
  ips.ipad,
  -- Moradia
  ips.dcra,
  ips.diea,
  ips.dpa,
  ips.dpsa,
  -- Segurança Pessoal
  ips.aj,
  ips.am,
  ips.homicidios,
  ips.mat,
  -- Acesso ao Conhecimento Básico
  ips.aef,
  ips.aem,
  ips.eem,
  ips.disem,
  ips.ideb_ef,
  ips.ref,
  -- Acesso à Informação e Comunicação
  ips.cim,
  ips.di_blf,
  ips.dtm,
  ips.qim,
  -- Saúde e Bem-estar
  ips.ev,
  ips.m15_50,
  ips.mdcnt,
  ips.obesidade,
  ips.suicidios,
  -- Qualidade do Meio Ambiente
  ips.avu,
  ips.eco2h,
  ips.fc,
  ips.ivcm,
  ips.svps,
  -- Direitos Individuais
  ips.apdh,
  ips.eadmn,
  ips.iadj,
  ips.tclp,
  -- Liberdades Individuais e de Escolha
  ips.acle,
  ips.ga19,
  ips.ppau,
  ips.ti,
  -- Inclusão Social
  ips.pgcm,
  ips.pnpcm,
  ips.vci,
  ips.vcn,
  ips.vcm,
  -- Acesso à Educação Superior
  ips.ees,
  ips.mees
from enem
inner join `brasil-aberto-443222.microdados_abertos.municipios_ips` as ips
    on (
        ips.codigo = cast(enem.`CO_MUNICIPIO_ESC` as string)
    )
"""

municipalities_df = client.query(query).to_dataframe()
municipalities_df.head()

,nota_media,pop,pib_pc,cv_polio,hcsap,macsap,mi5,subnutricao,aavrd,esa,...,ga19,ppau,ti,pgcm,pnpcm,vci,vcn,vcm,ees,mees
0,2424.829412,10392,49662.545891,33.179724,358.306189,169.509361,10.989011,3.942308,34.243243,9.547546,...,21.978022,0.000000,45.529659,0.699233,0.906998,238.095238,39.525692,222.042794,30.726739,45.274883
1,2363.361538,43467,16497.130080,47.944007,791.218843,163.112025,28.492647,4.867987,38.011114,14.413228,...,30.238971,0.095039,3.295436,0.382420,0.869094,6.006006,117.994100,333.881738,36.185048,47.892110
2,2303.610000,9222,12449.253687,100.000000,139.082058,5.312809,43.478261,3.847073,33.063749,31.707317,...,34.347826,0.000000,4.746084,0.230107,0.637122,10.548523,184.162063,67.370312,50.928700,61.980831
3,2411.414062,18757,34613.592611,45.222930,1259.286580,139.349456,7.194245,5.315030,61.027062,76.199669,...,16.546763,0.696853,2.128112,0.446755,0.599144,0.000000,95.628415,21.436227,28.925229,39.458635
4,2502.544118,14021,28645.295806,74.319066,279.880415,189.170340,24.390244,4.111509,27.893245,83.907126,...,20.000000,0.000000,3.653652,0.000000,0.873607,55.555556,119.760479,334.935197,27.457583,32.993562


In [40]:
faixas_de_renda = municipalities_df["Q006"].unique()

In [41]:
dfs = {}

for faixa_de_renda in faixas_de_renda:
    dfs[faixa_de_renda] = municipalities_df[municipalities_df["Q006"] == faixa_de_renda]

In [ ]:
models = {}

for faixa_de_renda in faixas_de_renda:
    s = setup(dfs[faixa_de_renda], target="nota_media", session_id=123)

    print(f'Model: {faixa_de_renda}')
    models[faixa_de_renda] = create_model('rf')

ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).